In [4]:
import pandas as pd
from bs4 import BeautifulSoup

In [146]:
articles = pd.read_csv('./data/locations.csv')
articles = articles[articles['Children'] == False]
articles = articles[articles['Company'] != 'WMT']
articles = articles[articles['Company'] != 'GOOGL']
articles.dropna(inplace=True)

articles.sort_values(by=['Company', 'Date'], inplace=True)
articles.reset_index(drop=True, inplace=True)


In [172]:
def is_qa_session(s):
    if s == None:
        return False
    s1 = s.parent.text
    s1 = s1.strip().lower().replace('-', ' ') 
    if s1 == 'question and answer session':
        return True
    else:
        if s1 in ['question and', 'question']:
            p_s = s.parent.parent.text
            if p_s.strip().lower().replace('-', ' ') == 'question and answer session':
                return True

    return False

def find_tag(row):
    path = './data' + row['Path']
    article = open(path, 'r')
    soup = BeautifulSoup(article, 'html.parser')

    header = soup.find_all(['p', 'strong'], string=is_qa_session)
    if len(header)== 2:
        if header[0].text != header[1].text:
            print(path)
    if header == None:
        print(path)
    
    header = header[0]
    pre_count = 0
    next_count = 0
    pre = header.find_all_previous('p')
    for p in pre:
        pre_count += len(p.text)
    next = header.find_all_next('p')
    for n in next:
        next_count += len(n.text)

    return [pre_count, next_count]
    
word_count = articles.apply(find_tag, axis=1, result_type='expand')

word_count.rename(columns={0: 'speech_word', 1: 'qa_word'}, inplace=True)

df_temp = pd.concat([articles, word_count], axis=1)


In [173]:
df_temp.to_csv('./data/word_count_by_session.csv')